# Blue Brain BioExplorer

![](./images/bioexplorer_banner.png)

In [ ]:
from bioexplorer import BioExplorer, Protein, Surfactant, Membrane, Volume, \
                        Cell, Vector2, Vector3
import nglview

be = BioExplorer('localhost:5000')

In [ ]:
status = be.reset()

### Resources

In [ ]:
resource_folder = '../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'

glucose_path = pdb_folder + 'glucose.pdb'
lactoferrin_path=pdb_folder + 'immune/1b0l.pdb'
defensin_path = pdb_folder + 'immune/1ijv.pdb'

surfactant_head_source = pdb_folder + 'surfactant/1pw9.pdb'
surfactant_branch_source = pdb_folder + 'surfactant/1k6f.pdb'

### Configuration

In [ ]:
# Scene
scene_size = 250

# Immune system
nb_glucoses = 3600
nb_lactoferrins = 10
nb_defensins = 30

# Proteins
protein_radius_multiplier = 1.0
protein_representation = be.REPRESENTATION_ATOMS
protein_load_hydrogen = False

## Camera

In [ ]:
status = be.core_api().set_camera(
    orientation=[0.0, 0.0, 0.0, 1.0],
    position=[4.883842102781712, 44.255928195014, 431.9119490747046],
    target=[4.883842102781712, 44.255928195014, 31.311730081826504])

## Scene description

### Coronavirus

In [ ]:
name='Coronavirus'
be.add_coronavirus(name=name, resource_folder=resource_folder, 
                   representation=protein_representation,
                   add_glycans=True, position=Vector3(75, 0.0, 60.0),
                   clipping_planes=[[-1,0,-1,35]])

### Host cell

In [ ]:
# ACE2 receptor definition
ace2_receptor = Protein(
    sources=[pdb_folder + '6m1d.pdb'],
    number_of_instances=1,
    position=Vector3(0.0, 6.0, 0.0))

# Membrane definition
membrane_size = scene_size
membrane_height = 80
membrane = Membrane(
    sources=[pdb_folder + 'membrane/popc.pdb'],
    number_of_instances=30000)

# Cell definition
name='Cell'
cell = Cell(
    name=name,
    size=Vector2(membrane_size, membrane_height),
    shape=be.ASSEMBLY_SHAPE_SINUSOIDAL,
    membrane=membrane, receptor=ace2_receptor)

# Add cell to scene
status = be.add_cell(
    cell=cell, representation=protein_representation,
    atom_radius_multiplier=protein_radius_multiplier,
    position=Vector3(0.0, -80.0, 0.0))

status = be.set_protein_color_scheme(
    name, name + '_' + be.NAME_RECEPTOR, 
    be.COLOR_SCHEME_CHAINS, 'OrRd_r', 7)

### Environment

In [ ]:
volume_position = Vector3(0.0, scene_size / 2.0 - 80.0, 0.0)

#### Glucose

In [ ]:
nglview.show_file(glucose_path)

In [ ]:
glucose = Protein(
    sources=[glucose_path], load_non_polymer_chemicals=True, 
    number_of_instances=nb_glucoses)
volume = Volume(
    name=be.NAME_GLUCOSE, size=Vector2(scene_size, scene_size), protein=glucose)
be.add_volume(
    volume=volume, 
    representation=protein_representation,
    atom_radius_multiplier=protein_radius_multiplier,
    position=volume_position)

#### Lactoferrin

In [ ]:
nglview.show_file(lactoferrin_path)

In [ ]:
lactoferrin = Protein(
    sources=[lactoferrin_path], load_non_polymer_chemicals=True,
    number_of_instances=nb_lactoferrins)
lactoferrins_volume = Volume(
    name=be.NAME_LACTOFERRIN, size=Vector2(scene_size, scene_size), protein=lactoferrin)
be.add_volume(
    volume=lactoferrins_volume,
    representation=protein_representation,
    atom_radius_multiplier=protein_radius_multiplier,
    position=volume_position)

#### Defensins

In [ ]:
nglview.show_file(defensin_path)

In [ ]:
defensin = Protein(
    sources=[defensin_path], load_non_polymer_chemicals=True, 
    number_of_instances=nb_defensins)
defensins_volume = Volume(
    name=be.NAME_DEFENSIN, size=Vector2(scene_size, scene_size), protein=defensin)
be.add_volume(
    volume=defensins_volume, 
    representation=protein_representation,
    atom_radius_multiplier=protein_radius_multiplier,
    position=volume_position)

#### Surfactant-D

In [ ]:
def add_surfactant_d(name, position, random_seed):
    surfactant_d = Surfactant(
        name=name, surfactant_protein=be.SURFACTANT_PROTEIN_D, 
        head_source=surfactant_head_source,
        branch_source=surfactant_branch_source)
    be.add_surfactant(
        surfactant=surfactant_d, 
        representation=protein_representation,
        atom_radius_multiplier=protein_radius_multiplier,
        position=position, random_seed=random_seed)

add_surfactant_d(
    name='Surfactant-D 1', position=Vector3(2.0, 18.0, 102.0), random_seed=1)

#### Surfactant A

In [ ]:
def add_surfactant_a(name, position, random_seed):
    surfactant_a = Surfactant(
        name=name, surfactant_protein=be.SURFACTANT_PROTEIN_A, 
        head_source=surfactant_head_source,
        branch_source=surfactant_branch_source)
    be.add_surfactant(
        surfactant=surfactant_a, 
        representation=protein_representation,
        atom_radius_multiplier=protein_radius_multiplier,
        position=position, random_seed=random_seed)

add_surfactant_a(
    name='Surfactant-A 1', position=Vector3(-100.0, 0.0, 0.0), random_seed=2)

## Materials

In [ ]:
be.apply_default_color_scheme(
    specular_exponent=5, glossiness=1,
    user_parameter=0.5,
    shading_mode=be.SHADING_MODE_ELECTRON)

## Rendering settings

In [ ]:
be.core_api().set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer',
    samples_per_pixel=1, subsampling=4, max_accum_frames=64)
params = be.core_api().BioExplorerRendererParams()
params.gi_samples = 3
params.gi_weight = 0.4
params.gi_distance = 500
params.shadows = 0.8
params.soft_shadows = 0.3
params.fog_start = 1500
params.fog_thickness = 1500
params.max_bounces = 3
be.core_api().set_renderer_params(params)